In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import heapq
from matplotlib.colors import ListedColormap
from IPython.display import HTML, display

# --------------------------
# 1. Grid setup (6x6)
# --------------------------
TAILLE_GRILLE = 6
grille = np.zeros((TAILLE_GRILLE, TAILLE_GRILLE))

# Obstacles positioned to make reflex agent take a longer path
grille[0, 1] = 1
grille[1, 2] = 1
grille[2, 2] = 1
grille[3, 1] = 1
grille[4, 3] = 1

# Goal
grille[TAILLE_GRILLE - 1, TAILLE_GRILLE - 1] = 2

start = (0, 0)
goal = (TAILLE_GRILLE - 1, TAILLE_GRILLE - 1)

# --------------------------
# 2. Reflex agent (always right → down)
# --------------------------
def reflex_agent_path(start, grid):
    path = []
    x, y = start
    while True:
        path.append((x, y))
        # Try moving right first, then down
        if y + 1 < TAILLE_GRILLE and grid[x, y + 1] != 1:
            y += 1
        elif x + 1 < TAILLE_GRILLE and grid[x + 1, y] != 1:
            x += 1
        else:
            break
        if (x, y) == goal:
            path.append((x, y))
            break
    return path

# --------------------------
# 3. Goal-based agent (A*)
# --------------------------
def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def goal_based_agent_path(start, goal, grid):
    frontier = []
    heapq.heappush(frontier, (0, start))
    came_from = {start: None}
    cost_so_far = {start: 0}

    while frontier:
        _, current = heapq.heappop(frontier)
        if current == goal:
            break
        for dx, dy in [(-1,0),(1,0),(0,-1),(0,1)]:
            nx, ny = current[0] + dx, current[1] + dy
            next_cell = (nx, ny)
            if 0 <= nx < TAILLE_GRILLE and 0 <= ny < TAILLE_GRILLE and grid[nx, ny] != 1:
                new_cost = cost_so_far[current] + 1
                if next_cell not in cost_so_far or new_cost < cost_so_far[next_cell]:
                    cost_so_far[next_cell] = new_cost
                    priority = new_cost + heuristic(goal, next_cell)
                    heapq.heappush(frontier, (priority, next_cell))
                    came_from[next_cell] = current

    # Reconstruct path
    current = goal
    path = []
    while current:
        path.append(current)
        current = came_from.get(current)
    path.reverse()
    return path

# --------------------------
# 4. Animation function
# --------------------------
def animate_agent(grid, path, agent_color_index, title):
    fig, ax = plt.subplots()
    display_grid = grid.copy()
    color_list = ['white', 'gray', 'green', 'red', 'blue']
    cmap = ListedColormap(color_list)

    def update(frame):
        ax.clear()
        ax.set_xticks(np.arange(TAILLE_GRILLE + 1) - 0.5, minor=True)
        ax.set_yticks(np.arange(TAILLE_GRILLE + 1) - 0.5, minor=True)
        ax.grid(which='minor', color='black', linestyle='-', linewidth=1)
        ax.tick_params(which='both', bottom=False, left=False, labelbottom=False, labelleft=False)

        current_grid = display_grid.copy()
        if frame < len(path):
            x, y = path[frame]
            current_grid[x, y] = agent_color_index

        ax.imshow(current_grid, cmap=cmap, vmin=0, vmax=len(color_list) - 1)
        ax.set_title(f"{title} - Step {frame+1}/{len(path)}")

    anim = animation.FuncAnimation(fig, update, frames=len(path), interval=700, repeat=False, blit=False)
    plt.close(fig)
    return anim

# --------------------------
# 5. Generate paths
# --------------------------
reflex_path = reflex_agent_path(start, grille)
goal_path = goal_based_agent_path(start, goal, grille)

# --------------------------
# 6. Display animations
# --------------------------
print("🔴 Reflex Agent vs 🔵 Goal-Based Agent (A*):")
display(HTML(animate_agent(grille, reflex_path, 3, "Reflex Agent").to_jshtml()))
display(HTML(animate_agent(grille, goal_path, 4, "Goal-Based Agent").to_jshtml()))

# --------------------------
# 7. Print path lengths
# --------------------------
print(f"Reflex Agent steps: {len(reflex_path)}")
print(f"Goal-Based Agent steps: {len(goal_path)}")


🔴 Reflex Agent vs 🔵 Goal-Based Agent (A*):


Reflex Agent steps: 4
Goal-Based Agent steps: 11
